In [176]:
import pandas as pd
import re
import math as math
history = pd.read_csv("../data/raw/history_full_data.csv")

In [177]:
import re
def extract_year_price(row):
    string = {}
    price_string = row['cost_text'].replace(",","")
    price_year = re.findall(r'\d+[$]\d+', price_string)
    for each_year in price_year:
        year = re.search(r'\d+[$]', each_year).group(0).replace("$","")
        price = re.search(r'[$]\d+', each_year).group(0).replace("$","")
        if price and year:
            string[int(year)] = int(price)    
    return string

In [178]:
history.apply(lambda row:extract_year_price(row),axis=1)

0                               {2021: 500, 2020: 550}
1                                          {2021: 390}
2                                          {2021: 645}
3                                          {2020: 430}
4                               {2019: 420, 2017: 410}
                             ...                      
33857     {2022: 410, 2019: 310, 2017: 295, 2015: 260}
33858                                      {2022: 550}
33859                                   {2022: 820000}
33860    {2022: 480, 2016: 300, 2014: 1300, 2013: 300}
33861                                  {2022: 1395000}
Length: 33862, dtype: object

In [179]:
history["price_in_history"] = history.apply(lambda row:extract_year_price(row),axis=1)

In [180]:
# counting the # of rows
for year in range(2017,2022):
    count = 0
    for index in range(33862):
        if year in history.iloc[index]['price_in_history']:
            count += 1
    print(f'{year}:{count}')

2017:4469
2018:4911
2019:5460
2020:5747
2021:6903


In [181]:
history['year'] = history.apply(lambda row:list(row['price_in_history'].items()),axis=1)

In [182]:
history = history.explode('year')

In [183]:
history = history[history['price_in_history']!={}]

In [184]:
history["years"]= history.apply(lambda row:row['year'][0],axis=1)
history["price"]= history.apply(lambda row:row['year'][1],axis=1)

In [185]:
history = history.drop(['Unnamed: 0', 'cost_text','price_in_history','year'], axis=1)

In [186]:
# overview about spliting the rent of each year
history['postcode'] = history['postcode'].astype('str')

# Data preprocessing 2017-2021

In [187]:
# getting the the table postcodes matches to SA2 region name
# The data scrape from https://www.matthewproctor.com/full_australian_postcodes_vic
import requests
from bs4 import BeautifulSoup
url = "https://www.matthewproctor.com/full_australian_postcodes_vic" #网址
data = requests.get(url).text  #获取数据
soup = BeautifulSoup(data, 'html.parser') #获取数据中想要的东西
table = soup.find("table", {"class": "table table-striped"}) #获取其中的表格
df = pd.DataFrame(columns=['postcodes', 'SA2_names'])

# Collecting Ddata
for row in table.find_all('tr'):    
    # Find all data for each column
    columns = row.find_all('td')
    
    if(columns != []):
        postcode = columns[1].text.strip()
        SA2_name = columns[19].text.strip()
        df = df.append({'postcodes': postcode,'SA2_names': SA2_name}, ignore_index=True)
sa2_dataframe = df.iloc[1:]
def add_sa2(row,sa2_dataframe):
    postcode = row['postcode']
    selected_postcode = sa2_dataframe.loc[sa2_dataframe['postcodes'] == postcode]
    sa2 = [(selected_postcode.iloc[i])["SA2_names"] for i in range(len(selected_postcode))]
    sa2=list(set(sa2))
    return sa2
history['sa2_region'] = history.apply(lambda row: add_sa2(row,sa2_dataframe), axis=1)

/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_1377/1042629386.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'postcodes': postcode,'SA2_names': SA2_name}, ignore_index=True)
/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_1377/1042629386.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'postcodes': postcode,'SA2_names': SA2_name}, ignore_index=True)
/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_1377/1042629386.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'postcodes': postcode,'SA2_names': SA2_name}, ignore_index=True)
/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_1377/1042629386.py:19: FutureWarning: The frame

In [188]:
SA2_centroids = pd.read_csv('../data/raw/mb_2016_centroids.csv')
all_value = []
SA2_vic = SA2_centroids[SA2_centroids['STE_NAME16']=='Victoria']
SA2_vic = SA2_vic[['SA2_MAIN16','SA2_NAME16','xcoord','ycoord']]
SA2_vic.head()
SA2_vic_xcoord = SA2_vic.groupby('SA2_NAME16')["xcoord"].mean()
SA2_vic_ycoord = SA2_vic.groupby('SA2_NAME16')["ycoord"].mean()
#现在拿到了所有SA2的中心区域
SA2_vic_xy = pd.merge(SA2_vic_xcoord,SA2_vic_ycoord,left_on = 'SA2_NAME16',right_on = 'SA2_NAME16')
#groupy the SA2_NAME16 and take the average of xcoord and ycoord
SA2_vic_xy=dict(SA2_vic_xy.transpose())
all_co = []
for value in history['sa2_region']:
    ind_co=[] #每一行的结果
    for region in value:
        #print(region)
        if region in SA2_vic_xy.keys():
            ind_co.append(SA2_vic_xy[region])
    all_co.append(ind_co)
history['sa2_coor'] = all_co

In [189]:
#function to calculate the distance 
def getDistance(latA, lonA, latB, lonB):
    ra = 6378140  # 赤道半徑
    rb = 6356755  # 極半徑
    flatten = (ra - rb) / ra  # Partial rate of the earth
    # change angle to radians
    radLatA = math.radians(latA)
    radLonA = math.radians(lonA)
    radLatB = math.radians(latB)
    radLonB = math.radians(lonB)

    pA = math.atan(rb / ra * math.tan(radLatA))
    pB = math.atan(rb / ra * math.tan(radLatB))
    x = math.acos(math.sin(pA) * math.sin(pB) + math.cos(pA) * math.cos(pB) * math.cos(radLonA - radLonB))
    c1 = (math.sin(x) - x) * (math.sin(pA) + math.sin(pB)) ** 2 / math.cos(x / 2) ** 2
    c2 = (math.sin(x) + x) * (math.sin(pA) - math.sin(pB)) ** 2 / math.sin(x / 2) ** 2
    dr = flatten / 8 * (c1 - c2)
    distance = ra * (x + dr)
    distance = round(distance / 1000, 4)
    return distance

In [190]:
# 找出离房子最近的sa2
def close_sa2(row): 
    all_distance = [getDistance(row['latitude'],row['longitude'],x[1],x[0]) for x in row.sa2_coor]
    min_distance = min(all_distance,default = 0)
    return row['sa2_region'][all_distance.index(min_distance)]   
history['closest_sa2'] = history.apply(lambda row:close_sa2(row),axis=1)

# Data preprocessing

In [191]:
#take the number of bedroom bathroom and car spaces 
import numpy as np
def data_preprocessing(dataset):
    bed_num=[]
    bath_num=[]
    park_num=[]
    #seperate the bedroom, bathroom and parking area
    for value in dataset['bed']:
        bed_num.append(re.findall(r'Bed : (\d+)', str(value))) #找到bed前面的数字
    for value in dataset['bath']:
        bath_num.append(re.findall(r'Bath : (\d+)', str(value)))
    for value in dataset['car']:
        park_num.append(re.findall(r'Car : (\d+)', str(value)))
    dataset['bed'] = bed_num
    dataset['bath'] = bath_num
    dataset['car'] = park_num
data_preprocessing(history)

In [193]:
def extract_bed(row):
    if row['bed'] == []:
        return None
    return row['bed'][0]
def extract_bath(row):
    if row['bath'] == []:
        return None
    return row['bath'][0]
def extract_car(row):
    if row['car'] == []:
        return None
    return row['car'][0]
history['bed'] = history.apply(lambda row:extract_bed(row),axis=1)
history['bath'] = history.apply(lambda row:extract_bath(row),axis=1)
history['car'] = history.apply(lambda row:extract_car(row),axis=1)

In [196]:
history.head()

,name,postcode,property_type,bed,bath,latitude,longitude,car,years,price,sa2_region,sa2_coor,closest_sa2
0,"609/1 BOUVERIE STREET, CARLTON SOUTH",3000,Category : Unit/apmt,3,2,-37.80610,144.96122,None,2021,500,[Melbourne],"[[144.9627648781132, -37.813376920301884]]",Melbourne
0,"609/1 BOUVERIE STREET, CARLTON SOUTH",3000,Category : Unit/apmt,3,2,-37.80610,144.96122,None,2020,550,[Melbourne],"[[144.9627648781132, -37.813376920301884]]",Melbourne
1,"507/22 ORR STREET, CARLTON SOUTH",3000,Category : Unit/apmt,2,2,-37.80634,144.96535,None,2021,390,[Melbourne],"[[144.9627648781132, -37.813376920301884]]",Melbourne
2,"112/1 QUEENSBERRY STREET, CARLTON SOUTH",3000,NaN,2,2,-37.80541,144.96875,1,2021,645,[Melbourne],"[[144.9627648781132, -37.813376920301884]]",Melbourne
3,"3203/488 SWANSTON STREET, CARLTON SOUTH",3000,NaN,2,1,-37.80558,144.96354,None,2020,430,[Melbourne],"[[144.9627648781132, -37.813376920301884]]",Melbourne


# Merge the 2017-2021 population and income into the dataset